# Atelier ReInvent 2019 Amazon Personalize

![MainDiagram](static/imgs/image.png)

## Ordre du jour

De manière générale, l'utilisation d'Amazon Personalize suit les étapes du schéma ci-dessous :

![FlowDiagram](static/imgs/personalize_process.png)

Le protocole à suivre est le suivant :

1. Importations et configuration
1. Préparation de vos données
1. Importation de vos données
1. Sélection d'une recette
1. Entraînement d'une solution
1. Déploiement d'une campagne
1. Obtention de recommandations
1. Interactions en temps réel
1. Conclusion
1. Bonus : Exportation en masse des recommandations

Dans chacune de ces étapes, vous verrez et exécuterez des extraits de code écrits en Python à l'aide de notre kit SDK Boto3. Ces extraits peuvent être modifiés pour devenir des composants de votre intégration de production avec Personalize. 

Vous trouverez dans ce bloc-notes les étapes de la création d'un modèle de recommandation de films adapté à des utilisateurs spécifiques, l'objectif étant de recommander des films en fonction de l'historique des interactions positives de ces utilisateurs avec les films.

Les données sont fournies par le [projet MovieLens](https://movielens.org). Vous pourrez en savoir plus à ce sujet plus tard si vous êtes intéressé.

Le contenu ci-dessous a été rédigé pour vous guider dans le processus au cours d'une séance d'atelier. Vous pouvez déployer ultérieurement le modèle CloudFormation que vous avez trouvé sur GitHub sur votre propre compte et vous pouvez parcourir les bloc-notes 1 à 3 pour refaire le même exercice. En outre, vous pouvez mettre à jour le contenu des cellules pour prendre en compte vos propres données. Il s'agit d'un moyen efficace de créer un modèle de recommandation personnalisé pour votre cas d'utilisation.

## Comment utiliser le bloc-notes

Le code est décomposé en cellules comme celle ci-dessous. Vous trouverez en haut de cette page un bouton `Run` triangulaire sur lequel vous pouvez cliquer pour valider chaque cellule et passer à la suivante. Vous pouvez aussi appuyer sur `Shift` + `Enter` ou `Shift` + `Return` (utilisateurs Mac) lorsque vous êtes dans la cellule pour la valider et passer à la suivante.


Lorsqu'une cellule est en cours d'exécution, vous remarquerez une ligne sur le coté affichant une `*` pendant que la cellule est opérationnelle ou elle se mettra à jour sous forme de nombre pour indiquer la dernière cellule qui a terminé l'exécution après avoir achevé l'exécution de tous les codes dans une cellule.


Il suffit de suivre les instructions ci-dessous et d'exécuter les cellules pour commencer avec Amazon Personalize.



## Importations et configuration 

Python est livré avec une grande collection de bibliothèques. Nous devons les importer ainsi que celles qui sont installées pour nous aider comme boto3 (le kit SDK AWS) et Pandas/Numpy qui sont des outils de base de la science des données. La cellule ci-dessous les importera pour les utiliser ici. Elle mettra également à jour la bibliothèque boto3 vers la dernière version. Vous pourriez voir apparaître un avertissement (texte jaune) ou une erreur (texte rouge). Ne vous inquiétez pas. Continuez simplement à exécuter la cellule suivante.

In [ ]:
# Update boto3
!pip install --upgrade boto3
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime
import uuid

Ensuite, vous devez vérifier que votre environnement peut communiquer avec Amazon Personalize. Les lignes ci-dessous servent à cela.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

La dernière étape consiste à déterminer la région dans laquelle vous organisez cet atelier. La cellule ci-dessous le fera et l'assignera à la variable `region`.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]

## Préparation de vos données

Tout d'abord, vous aurez besoin d'une collection de points de données où les utilisateurs ont interagi avec le contenu d'une manière ou d'une autre. Amazon Personalize part du principe que si une interaction est enregistrée, elle est positive. Vous verrez plus tard comment nous utilisons cet aspect. 

La cellule ci-dessous téléchargera les données dont vous avez besoin, extraira le contenu d'un fichier Zip, puis en affichera une infime partie sur votre écran.

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'], engine='python')
pd.set_option('display.max_rows', 5)
data

Comme vous pouvez le voir, les données contiennent un UserID, un ItemID, un Rating et un Timestamp.

Nous n'avons pas besoin de la colonne `Rating`. Elle sera donc supprimée avant de sauvegarder le fichier.

Une fois les données préparées, vous les enregistrerez localement dans un fichier CSV avec la dernière ligne.

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
filename = "movie-lens-ml-100k-prepared.csv"
data.to_csv(filename, index=False)

Avant de pouvoir télécharger les données dans S3, vous devez créer un compartiment, la cellule ci-dessous s'en chargera.

In [ ]:
print(region)
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "reinventpersonalizeworkshop"
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

Vous pouvez à présent télécharger le fichier sur S3 et il sera prêt à être importé dans Amazon Personalize.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

## Importation de vos données

Les étapes à suivre pour importer vos données sont les suivantes :

1. Créez un groupe de jeux de données.
1. Identifiez le schéma de votre jeux de données.
1. Créez un ensemble de données en utilisant votre schéma.
1. Exécutez un ImportJob pour charger les données à utiliser avec Personalize.


Dans Amazon Personalize, un groupe de jeux de données est la façon dont vos informations sont isolées de toute autre expérience. Aucune information n'est partagée entre ces groupes. Un groupe de jeux de données peut contenir vos données d'interaction, vos métadonnées sur les articles, vos métadonnées sur les utilisateurs, vos dispositifs de suivi des événements, vos solutions et vos campagnes. Pour en savoir plus à ce sujet : https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html 

Commencez par créer un groupe de jeux de données.

#### Créer un groupe de jeux de données

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-RI-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### Boucle de création d'un groupe de jeux de données

La création d'un DatasetGroup prend quelques secondes. La cellule ci-dessous sera donc interrogée jusqu'à ce que le DatasetGroup soit actif et que vous puissiez poursuivre l'atelier. Une fois que `ACTIVE` s'affiche, passez à l'étape `Create Schema`.


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(5)

#### Créer un schéma


Vous pouvez à présent déterminer le schéma de votre jeu de données. Compte tenu de la limite de temps, vous n'utiliserez dans cet atelier que les données relatives à l'interaction utilisateur-article, ou données d'interaction en abrégé. La cellule ci-dessous contient les articles exigés pour ce jeu de données et correspond à la structure du CSV que vous avez téléchargé précédemment.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

La cellule ci-dessous créera un schéma dans Amazon Personalize qui peut être connecté à un jeu de données ; c'est ainsi que Personalize interprète le contenu de votre fichier CSV.

In [ ]:
create_schema_response = personalize.create_schema(
    name = "personalize-ri-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

#### Créer un jeu de données


Ensuite, créez le jeu de données pour les interactions, attribuez-lui le schéma fourni et affectez-le au groupe de données que vous avez créé précédemment avec ce code :

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ri-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

#### Attacher une politique au compartiment S3

Avant de pouvoir exécuter la tâche d'importation de vos données, vous devrez joindre une politique de compartiment à votre compartiment S3 pour permettre à Personalize de communiquer avec lui, ainsi qu'un rôle IAM pour le service à utiliser dans ce compte AWS.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

Notez que les changements IAM dans la cellule ci-dessous ne prennent que quelques secondes pour être confirmés. Cette section prendra donc une minute pour être terminée. Exécutez ensuite la cellule ci-dessous.

#### Créer un rôle Personalize

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRIDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

Enfin, vous êtes prêt à importer les données dans Personalize. La première cellule ci-dessous lancera le processus et la seconde contient une boucle while qui interrogera le service pour déterminer quand le processus d'importation sera terminé. 

Au cours de cet atelier, vous allez importer un jeu de données relativement petit et cette opération peut sembler longue pour un si petit fichier. Dans ce cas, la priorité est donnée au provisionnement en ressources dédiées qui exécuteront réellement la tâche. Personalize est ainsi en mesure d'assurer la conformité HIPAA, par exemple. De plus, ce n'est pas parce que vos fichiers sont plus volumineux que l'importation est extrêmement longue : une fois les ressources en place, l'importation est assez rapide.

Le processus d'importation peut prendre jusqu'à 20 minutes. 

#### Créer une tâche d'importation de jeu de données

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ri-import",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### Boucle d'importation de jeux de données

Le remplissage de la cellule ci-dessous prendra un peu plus de temps. Il s'agit d'un sondage pour savoir quand votre jeu de données a été entièrement importé dans Personalize. Cette opération devrait prendre environ 20 minutes. La plus grosse partie du temps est consacrée au provisionnement de l'infrastructure en coulisse. Autrement dit, l'importation de fichiers beaucoup plus volumineux ne prend pas beaucoup plus de temps dans la plupart des cas.

Exécutez la cellule ci-dessous, puis passez à autre chose lorsqu'elle atteint un statut `ACTIVE`.


In [ ]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

## Sélection d'une recette

Avec Personalize, les algorithmes disponibles sont appelés des recettes. Le code ci-dessous vous en donne la liste complète. Ils présentent chacun des divers cas d'utilisation et vous trouverez des informations détaillées ici : https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html.

Dans cet atelier, vous utiliserez HRNN (Hierarchical Recurrent Neural Network) en citant les documents :

```
Le HRRN est un réseau neuronal récurrent hiérarchique, capable de modéliser les interactions utilisateur-article sur une période donnée. Utilisez la recette HRNN lorsque le comportement de l'utilisateur change au fil du temps, un phénomène que l'on appelle problème de l'intention évolutive.

Pour entraîner un modèle, HRNN utilise le jeu de données des interactions d'un groupe de jeu de données. Un groupe de jeux de données est un ensemble de jeux de données apparentés qui peuvent inclure les jeux de données Utilisateurs, Articles et Interactions.
```

Vous trouverez ici un document qui l'explique plus en détail en relation avec les recommandations : https://openreview.net/pdf?id=ByzxsrrkJ4. 


Il sert ici d'exemple de système de recommandation construit à l'aide de réseaux neuronaux profonds et peut être entraîné en utilisant uniquement nos données d'interactions. Il s'agit là d'un excellent point de départ pour mener vos propres expériences par la suite.

In [ ]:
personalize.list_recipes()

La cellule ci-dessous sélectionne la recette `HRNN`.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

## Entraînement d'une solution

Dans Amazon Personalize, un modèle entraîné sur des données client s'appelle une solution. Ces modèles sont classés par version et vous devez donc d'abord créer une solution, puis une version de solution. Les versions sont utilisées pour suivre l'amélioration du modèle sur une période donnée, en fonction des nouvelles données disponibles. 

La création d'une solution en soi est presque instantanée, mais l'entraînement à la création d'une version peut prendre un peu de temps. Il s'agit généralement de la période d'attente la plus longue de la procédure. Si vous vous livrez à cette expérience en dehors d'un atelier, c'est le moment idéal pour lire vos courriels, prendre un café, etc.

#### Créer une solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-ri-soln-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Créer une version de solution

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### Créer la boucle de la version de la solution

L'entraînement d'un modèle peut prendre un peu de temps. Il faut souvent compter au moins 20 minutes. Exécutez la cellule ci-dessous, en attendant à nouveau qu'elle passe à un état `ACTIVE` avant de passer à la prochaine étape.

In [ ]:
current_time = datetime.datetime.now()
print("Training Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Training Completed on: ", current_time.strftime("%I:%M:%S %p"))

#### Obtenir les métriques sur la version de la solution

Maintenant que votre solution et sa version sont en place, vous pouvez obtenir les métriques qui vous permettront de juger de ses performances.

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

## Créer et attendre la campagne

Maintenant que vous avez une version de la solution opérationnelle, vous devez créer une campagne pour l'utiliser avec vos applications. Une campagne est simplement une copie hébergée de votre modèle. Encore une fois, il faudra attendre un peu pour qu'après l'exécution, vous puissiez faire une petite pause pendant que l'infrastructure est mise en place.

#### Créer une campagne

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-ri-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Attendre que la campagne ait le statut ACTIF

#### Créer une boucle de campagne

Dans cette section, Personalize déploie votre modèle, ce qui prend quelques minutes. Exécutez la cellule ci-dessous pour interroger à nouveau jusqu'à ce que la tâche soit terminée. Cette opération devrait prendre 10 à 15 minutes. Une fois que la cellule a atteint l'état `ACTIVE`, continuez.

In [ ]:
current_time = datetime.datetime.now()
print("Deploying Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Deploying Completed on: ", current_time.strftime("%I:%M:%S %p"))

## Obtenir des exemples de recommandations

Une fois la campagne active, vous êtes prêt à recevoir des recommandations. Tout d'abord, nous devons sélectionner un utilisateur aléatoire à partir de la collection. Ensuite, nous créerons quelques fonctions d'aide pour obtenir des informations sur les films à afficher pour les recommandations au lieu de simples ID.

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE']

#### Appeler GetRecommendations

Le code ci-dessous obtient des recommandations pour l'utilisateur sélectionné au hasard. Il place ensuite les recommandations dans un cadre de données et les affiche.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Original Recommendations'])
recommendations_df

## Création d'un outil de suivi des événements

Pour que votre système de recommandation puisse répondre à des événements en temps réel, vous avez besoin d'un outil de suivi des événements. Le code ci-dessous en génère un et peut être utilisé pour la suite de cet atelier. N'hésitez pas à remplacer son nom par un nom plus descriptif.

In [ ]:
response = personalize.create_event_tracker(
    name='MovieClickTrackerRI',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

In [ ]:
event_tracker_arn = response['eventTrackerArn']

## Simulation du comportement des utilisateurs

Les lignes ci-dessous fournissent un exemple de code qui simule un utilisateur interagissant avec un article donné. Vous obtiendrez alors des recommandations différentes de celles de départ.

In [ ]:
session_dict = {}

In [ ]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
    )

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the title of the movie for the header of the column
    movie_title_clicked = get_movie_title(movie_to_click)
    # Interact with the movie
    send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)
    # Sleep for 2 seconds
    time.sleep(2)
    # Get new recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        title = get_movie_title(item['itemId'])
        recommendation_list.append(title)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])
    # Add this dataframe to the old one
    recommendations_df = recommendations_df.join(new_rec_DF)
    return recommendations_df

Les trois cellules ci-dessous simulent l'ajout d'un film au jeu de données et affichent les résultats après chaque interaction. La première colonne correspond aux recommandations originales, les colonnes suivantes correspondent à chaque film avec lequel l'utilisateur a interagi. L'en-tête de colonne est le nom du film avec lequel l'utilisateur a interagi, puis vous pourrez voir comment les recommandations sont modifiées.

In [ ]:
movie_to_click = 180
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

In [ ]:
movie_to_click = 210
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

In [ ]:
movie_to_click = 415
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

## Conclusion

Dans cet atelier, vous avez réussi à créer un groupe de jeux de données et un jeu de données basé sur les interactions. Vous avez aussi appris à entraîner un modèle de recommandation basé sur les données, à déployer une campagne pour générer des recommandations, à évaluer les recommandations initiales, à exploiter le suivi des événements en temps réel pour obtenir des recommandations encore meilleures. Vous trouverez ci-dessous des documents bonus montrant comment exporter les recommandations dans le système de gestion de l'information.  

Ce contenu restera public sur GitHub et pourra être utilisé au sein de votre organisation pour construire des modèles de recommandation personnalisés. Jetez un coup d'œil aux divers autres bloc-notes à l'avenir.

Merci, et bonne chance pour votre prochain projet de machine learning !

VOUS TROUVEREZ CI-DESSOUS LES DOCUMENTS BONUS :

## Recommandations par lots

Jusqu'à présent, vous avez appris à générer des recommandations au moyen d'un appel d'API et à interagir avec des systèmes de suivi des événements. Cette approche fonctionne bien pour de nombreuses applications, mais il se peut que vous souhaitiez mettre en cache localement toutes les recommandations destinées aux utilisateurs ou même étudier les recommandations pour trouver de nouvelles idées. Pour y parvenir, Amazon Personalize prend également en charge l'exportation par lots de vos recommandations vers un fichier. Les cellules ci-dessous vous guideront dans la soumission de recommandations à un fichier dans S3 et vous afficheront ensuite son contenu. Le fichier peut également être téléchargé de S3 vers votre ordinateur local.

Tout d'abord, vous devrez créer un fichier JSON de l'utilisateur IDS

In [ ]:
user_IDs = ['561', '233', '579']

json_input_filename = "json_input.json"
with open(json_input_filename, 'w') as json_input:
    for user_id in user_IDs:
        json_input.write('{"userId": "' + user_id + '"}\n')

Téléchargez à présent ce fichier sur S3 :

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

Maintenant que le fichier d'entrée est disponible dans S3, vous devez définir l'emplacement de la sortie et créer une tâche d'inférence par lots.

In [ ]:
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

La cellule ci-dessous crée la tâche de lot.

In [ ]:
personalize_rec = boto3.client(service_name='personalize')
batchInferenceJobArn = personalize_rec.create_batch_inference_job (
    solutionVersionArn = solution_version_arn,
    jobName = "RI-Workshop-Batch-Inference-Job",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

La cellule suivante procède à l'interrogation jusqu'à ce que l'exportation soit terminée. Cette boucle d'attente est la dernière de l'atelier ! Cette dernière va à nouveau interroger jusqu'à ce qu'elle indique l'état `ACTIVE`. Vous pouvez continuer une fois que cet état est atteint.

In [ ]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize_rec.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Une fois les données correctement exportées, récupérez le fichier et analysez-le.

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the movies
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            title = get_movie_title(item)
            recommendation_list.append(title)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

Dans la cellule ci-dessus, vous pouvez voir les différentes recommandations pour les utilisateurs fournies. Dans un scénario réel, la liste des utilisateurs pourrait être l'ensemble de votre base d'utilisateurs, vous permettant ainsi de référencer et de comparer rapidement les résultats entre eux.

## Bonus supplémentaire : Nettoyer

Les cellules ci-dessous sont absolument facultatives si vous utilisez Event Engine, dans la mesure où votre compte a été supprimé après l'atelier. Si vous souhaitez l'utiliser plus tard dans votre propre compte, les cellules ci-dessous supprimeront les ressources créées pour éviter des frais supplémentaires.

In [ ]:
# Delete the campaign:
personalize.delete_campaign(campaignArn=campaign_arn)
time.sleep(60)

In [ ]:
# Delete the solution
personalize.delete_solution(solutionArn=solution_arn)
time.sleep(60)

In [ ]:
# Delete the event tracker
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
time.sleep(60)

In [ ]:
# Delete the interaction dataset
personalize.delete_dataset(datasetArn=dataset_arn)
time.sleep(60)

In [ ]:
# Delete the event dataset
event_interactions_dataset_arn = dataset_arn
event_interactions_dataset_arn = event_interactions_dataset_arn.replace("INTERACTIONS", "EVENT_INTERACTIONS")
personalize.delete_dataset(datasetArn=event_interactions_dataset_arn)
time.sleep(60)

In [ ]:
# Delete the schema
personalize.delete_schema(schemaArn=schema_arn)

In [ ]:
# Delete the Dataset Group
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

In [ ]:
# Empty the S3 Bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
bucket.objects.all().delete()
time.sleep(60)

In [ ]:
# Delete the S3 Bucket
bucket.delete()

In [ ]:
# IAM policies should also be removed
iam = boto3.client("iam")
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess", RoleName=role_name)
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess",RoleName=role_name)

iam.delete_role(RoleName=role_name)

## Dernière étape

Après avoir supprimé toutes les ressources, vous pouvez fermer cette fenêtre et retourner à la page github sur laquelle vous vous trouviez. Au bas du fichier Readme se trouvent des étapes pour supprimer la pile CloudFormation que vous avez créée précédemment. Une fois cette étape franchie, vous avez terminé l'atelier à 100 %.

Félicitations !